# Othello Move Predictor

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
from pathlib import Path
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
from fastai.callbacks.mem import PeakMemMetric
from Othello_CNN import OthelloCNN
from torchsummary import summary

In [ ]:
# Redefine open_image
def open_image(fn:PathOrStr, div:bool=True, convert_mode:str='RGB', cls:type=Image,
        after_open:Callable=None)->Image:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning) # EXIF warning from TiffPlugin
        x = PIL.Image.open(fn).convert(convert_mode)
    if after_open: x = after_open(x)
    x = pil2tensor(x,np.float32)
#     if div: x.div_(255)
    return cls(x)

fastai.vision.Image.open_image = open_image

In [ ]:
# Batch size
bs = 256

# Define path to images
data_path = Path( '' )

# Get a list of names of all images in folder
fnames = get_image_files( data_path )

# Define function to retrieve label from filename
def get_labels(file_path):
    file_name = file_path.name   # Get the file name (not path)
    label = file_name[:2]        # Get the first two chars of name

# Load data into Databunch
data = ImageDataBunch.from_name_func( data_path, fnames, label_func=get_labels, bs=bs)

data.show_batch( rows = 3 )

In [ ]:
# Load model onto GPU..?
device = torch.device('cude')
model = OthelloCNN().to(device)

# Create a learner
learn = cnn_learner( data, model, metrics = error_rate )

# Find the optimal learning rate
lr_find( learn )
learn.recorder.plot()

In [ ]:
lr = 1e-3

learn.fit_one_cycle( 4, lr )

learn.save( 'stage1' )
learn.load( 'stage1' )